In [1]:
print("hello")

hello


In [ ]:
import numpy as np
from numpy import pi, log


#some constants
h = 0.678 #Gaia 2016

G = 6.67430e-11

Msun = 1.98847e30
kpc = 3.0856775814913673e19


rhocrit = 3*(100*h/(kpc))**2/(8*np.pi*G) * kpc**3 / Msun

#NFW  density
def rhoNFW(r,rhos,rs):
    return rhos/((r/rs)*(1.+(r/rs))**2.)


fac = 102

def delc(c):
    return (fac/3)*c**3/(log(1+c)-c/(1+c))
    
#NFW density (virial mass in Msun, concentration parameter)
def rho_nfw(r, mvir, cvir):
    
    rv = (mvir/((4/3) * pi * fac * rhocrit))**(1/3) 
    
    rs = rv / cvir
    
    rhos = rhocrit*delc(cvir)
    
    return rhoNFW(r, rhos, rs)

#Enclosed mass (virial mass in Msun, concentration parameter) in Msun
def mass_nfw(r, mvir, cvir):
    
    rv = (mvir/((4/3) * pi * fac * rhocrit))**(1/3)
    
    rs = rv / cvir
    
    rhos = rhocrit*delc(cvir)
    
    return 4 * pi *  rhos * rs**3 * (log((r + rs) / rs) + rs/(r + rs) - 1) 